In [ ]:
from pathlib import Path
import pandas as pd
from pgscen.command_line import (
    load_load_data, split_actuals_hist_future, split_forecasts_hist_future)


cur_path = Path("day-ahead_load_scenario.ipynb").parent.resolve()
data_dir = Path(cur_path, '..', "data").resolve()
load_zone_actual_df, load_zone_forecast_df = load_load_data()

scenario_start_time = pd.to_datetime('2018-02-21 06:00:00',utc=True)
scen_timesteps = pd.date_range(start=scenario_start_time,
                               periods=24, freq='H')

(load_zone_actual_hists,
     load_zone_actual_futures) = split_actuals_hist_future(
            load_zone_actual_df, scen_timesteps)

(load_zone_forecast_hists,
     load_zone_forecast_futures) = split_forecasts_hist_future(
            load_zone_forecast_df, scen_timesteps)

In [ ]:
from pgscen.engine import GeminiEngine

scenario_count=1000

ge = GeminiEngine(load_zone_actual_hists, load_zone_forecast_hists,
                  scenario_start_time, asset_type='load')

ge.fit(5e-2, 5e-2)
ge.create_scenario(scenario_count, load_zone_forecast_futures,
                   bin_width_ratio=0.1, min_sample_size=400)

In [ ]:
ge.forecasts['load']['North']

In [ ]:
df = ge.model.deviation_dict['North']
cap = df['Forecast'].max()

In [ ]:
(df['Forecast']-1698.4282).abs().sort_values().index[0:200]

In [ ]:
df.iloc[[100, 101]]['Actual']

In [ ]:
index = (df['Forecast']-1698.4282).abs().sort_values().index[0:200]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [19, 11]

_ = plt.hist(df.loc[index]['Deviation'], bins=30)

In [ ]:
plt_asset, plt_ts = tuple(ge.model.conditional_gpd_dict.keys())[11]
print('   '.join([plt_asset, str(plt_ts)]))

_ = plt.hist(ge.model.conditional_gpd_dict[plt_asset, plt_ts].slots['data'],
             bins=30)

In [ ]:
for col in ge.model.scen_df['Coast'].columns:
    plt.figure()
    _ = plt.hist(ge.model.scen_deviation_df['North'][col], bins=30)

In [ ]:
out_dir = Path(data_dir, '..')
ge.write_to_csv(out_dir, load_zone_actual_futures, write_forecasts=True)
!ls {Path(out_dir, "20180221")}

In [ ]:
import numpy as np

df = pd.read_csv("{}/20180221/load/South.csv".format(out_dir))
idx = np.random.randint(1000)
plt.plot(df.iloc[2+idx,2:],label='scen')
plt.plot(df.iloc[0,2:],label='act')
plt.plot(df.iloc[1,2:],label='fcst')
_ = plt.legend()